In [1]:
import json
import requests
import urllib.request
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 100)

import warnings
warnings.filterwarnings("ignore")

In [3]:
class ClinicalTrialsGovDocuments():
    def __init__(self):
        self.base_url_study_fields = 'https://clinicaltrials.gov/api/query/study_fields?'
        self.base_url_full_studies = 'https://clinicaltrials.gov/api/query/full_studies?'
        self.output_file_path = r"c:\\"

    def search(self, expr, fields):
        expr = expr.replace(' ', '+')
        url = self.base_url_study_fields + 'expr=' + expr + '&fields=' + fields + '&min_rnk=1&max_rnk=1000&fmt=json'
        urlf = self.base_url_full_studies + 'expr=' + expr + '&fmt=json'

        # search for specified database with the text string.
        # For each record found return the document.

        result = urllib.request.urlopen(url)
        text = result.read().decode('utf-8')

        json_text = json.loads(text)

        # Call retrieve_document() for each NCTid.

        study_fields = json_text["StudyfieldsResponse"]["StudyFields"]

        document_list = []
        for study_field in study_fields:
            document_list.append(study_field)

        return document_list
    
    def extract_single_values(self, df):
        nctids = []
        brief_titles = []
        conditions = []
        brief_summaries = []
        detailed_descriptions = []
        for _, row in df.iterrows():
            nctids.append(row.NCTId[0])
            brief_titles.append(row.BriefTitle[0])
            conditions.append(row.Condition[0])
            brief_summaries.append(row.BriefSummary[0])
            detailed_descriptions.append(row.DetailedDescription[0] if row.DetailedDescription != [] else "None")
        return pd.DataFrame({
            "NCTId": nctids,
            "BriefTitle": brief_titles,
            "Condition": conditions,
            "BriefSummary": brief_summaries,
            "DetailedDescription": detailed_descriptions
        })

    def drill_down_documents(self, df):
        nctids = []
        brief_titles = []
        conditions = []
        brief_summaries = []
        detailed_descriptions = []
        for _, row in df.iterrows():
            nctid = row.NCTId
            brief_title = row.BriefTitle
            condition = row.Condition
            brief_summary = row.BriefSummary
            detailed_description = row.DetailedDescription

            # breaking up rows with multiple conditions
            for doc_cond in row.Condition:
                nctids.append(nctid[0])
                brief_titles.append(row.brief_title[0])
                conditions.append(doc_cond)
                brief_summaries.append(brief_summary[0])
                detailed_descriptions.append(detailed_description[0] if detailed_description != [] else "None")
        return pd.DataFrame({
            "NCTId": nctids,
            "BriefTitle": brief_titles,
            "Condition": conditions,
            "BriefSummary": brief_summaries,
            "DetailedDescription": detailed_descriptions
        })
        

In [4]:
def create_clickable_id(id):
    url_template = '<a href="https://clinicaltrials.gov/ct2/show/{id}" target="_blank">{id}</a>'''.format(id=id)
    return url_template


In [5]:
ctd = ClinicalTrialsGovDocuments()

In [ ]:
fields = "NCTId,BriefTitle,Condition,BriefSummary,DetailedDescription"
document_list = pd.DataFrame(document_list).drop(['Rank'], axis=1)
df_documents = ctd.extract_single_values(df_documents)
df_documents['NCTId'] = df_documents['NCTId'].apply(create_Clickable_id)
df_docmuents = HTML(df_documents.to_html(index-False, render_links=True,escape=False))
df_documents

In [ ]:
# TODO break up search text into sentences and run each one through NLP
search_results = search_text.split('. ')

In [ ]:
search_results

In [ ]:
#search_text = "Are there cases in the last decade where patients with pericardial aortic valves inserted in the reverse position, to serve as mitral valve replacements."
#search_text = "smoking causes lung cancer"

#search_text = "High volume disease is defined as the presence of either visceral metastases or >= 4 bone metastases with >= 1 beyond the vertebral bodies and pelvis."
#search_text = "Sponsor submits certification"